In [1]:
import sys
sys.path.insert(0, '/home/johnr_000/jupyter_blog/jupyter_blog/scripts')
import eia_model as em
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from datetime import datetime

In [19]:
#variables to specify date range and frequency
api_key = em.eia_api
start = '2015-12-01 01:00:00'
end = '2016-01-31 23:00:00'
freq = 'H'
# create list of series used for dictionary keys
keys = ['EBA.BPAT-ALL.D.H', 'EBA.PACE-ALL.D.H', 'EBA.PACW-ALL.D.H', 
        'EBA.PGE-ALL.D.H', 'EBA.PSEI-ALL.D.H', 'EBA.SCL-ALL.D.H']
# create dict of GetSeriesRange objects from series list
series_dict = {key: em.GetSeries(api_key=api_key, series_id=key, 
                                   start=start, end=end, freq=freq) for key in keys}

In [20]:
# loop over dictionary and print out full name of each series
for key in series_dict:
    print('{}: {}'.format(series_dict[key].data.series[0]['name'],key))

Demand for PacifiCorp West (PACW), Hourly: EBA.PACW-ALL.D.H
Demand for Seattle City Light (SCL), Hourly: EBA.SCL-ALL.D.H
Demand for Portland General Electric Company (PGE), Hourly: EBA.PGE-ALL.D.H
Demand for Puget Sound Energy, Inc. (PSEI), Hourly: EBA.PSEI-ALL.D.H
Demand for Bonneville Power Administration (BPAT), Hourly: EBA.BPAT-ALL.D.H
Demand for PacifiCorp East (PACE), Hourly: EBA.PACE-ALL.D.H


In [21]:
# all the dataframes have the same date range so we can concatenate in a loop
concat_df = pd.concat([series_dict[key].data.df for key in series_dict], axis=1)
# and output as a pickle file - I'm doing this to avoid calling the EIA API as I develop
concat_df.to_pickle('data/nw_load_df.pkl')
# concat_df = pd.read_pickle('data/new_load_df.pkl')

In [2]:
concat_df = pd.read_pickle('/home/johnr_000/jupyter_blog/jupyter_blog/data/nw_load_df.pkl')

In [57]:
concat_df.describe()

/home/johnr_000/venv_python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,EBA.PACW-ALL.D.H,EBA.SCL-ALL.D.H,EBA.PGE-ALL.D.H,EBA.PSEI-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.PACE-ALL.D.H,hour,HOUR
count,1.487000e+03,1487.000000,1487.000000,1482.000000,1487.000000,1.487000e+03,1487.000000,1487.000000
mean,4.451198e+03,1282.258911,2579.921991,3946.066127,7059.477471,-2.470617e+04,11.507734,11.507734
std,6.558528e+04,177.394473,385.764476,554.268601,833.190306,8.950455e+05,6.920412,6.920412
min,-6.785000e+03,868.000000,1730.000000,2292.000000,5006.000000,-3.199017e+07,0.000000,0.000000
25%,2.393500e+03,1128.500000,2272.000000,NaN,6451.500000,5.169000e+03,6.000000,6.000000
50%,2.709000e+03,1324.000000,2646.000000,NaN,7109.000000,5.508000e+03,12.000000,12.000000
75%,2.941500e+03,1416.000000,2872.500000,NaN,7637.500000,5.822500e+03,17.500000,17.500000
max,2.530003e+06,1654.000000,3472.000000,5088.000000,9365.000000,5.966800e+04,23.000000,23.000000


In [9]:
concat_df['HOUR'] = concat_df.index.to_datetime().hour
# train, test = train_test_split(concat_df, test_size = 0.2)
# print('Length of training dataframe: {} and testing dataframe {}'.format(len(train), len(test)))

Length of training dataframe: 1189 and testing dataframe 298


In [59]:
rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=100, random_state=rng)
x_train = concat_df[['HOUR', 'EBA.PACW-ALL.D.H']].values
clf.fit(x_train)
y_pred_train = clf.predict(x_train)

In [60]:
test = clf.decision_function(x_train)

In [61]:
test2 = np.column_stack((concat_df.index.values, concat_df['EBA.PACW-ALL.D.H'].values, test))
for i in test2:
    print(i)

['2015-12-01 01:00:00' 3353 -0.08483705118914087]
['2015-12-01 02:00:00' 3468 -0.07716903753667825]
['2015-12-01 03:00:00' 3438 -0.07095122112248398]
['2015-12-01 04:00:00' 3357 -0.052216083054078655]
['2015-12-01 05:00:00' 3249 -0.0215427352310974]
['2015-12-01 06:00:00' 3077 0.02111862350666094]
['2015-12-01 07:00:00' 2829 0.04049894325177472]
['2015-12-01 08:00:00' 2648 0.05240513840540828]
['2015-12-01 09:00:00' 2538 0.05795174488755972]
['2015-12-01 10:00:00' 2494 0.05664477487348818]
['2015-12-01 11:00:00' 2505 0.05790208055768109]
['2015-12-01 12:00:00' 2545 0.05639834003252725]
['2015-12-01 13:00:00' 2644 0.05126887843674277]
['2015-12-01 14:00:00' 2937 0.033498391469789]
['2015-12-01 15:00:00' 3330 -0.03948369988206102]
['2015-12-01 16:00:00' 3533 -0.07207060364255036]
['2015-12-01 17:00:00' 3439 -0.054708045233725855]
['2015-12-01 18:00:00' 3363 -0.0383144664370364]
['2015-12-01 19:00:00' 3219 -0.002699216078814737]
['2015-12-01 20:00:00' 3093 0.022338274760597743]
['2015-12-

In [55]:
test2.shape

(1487, 2)